In [2]:
import requests
from bs4 import BeautifulSoup
import time, random
import json
import os

In [3]:
print(os.getcwd())

d:\work\myrepo


In [4]:
my_headers = [
    "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
    "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
    "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 "
]

In [9]:
def get_url_list():
  page_html = requests.get('https://vin17.com/',headers={'User-Agent':random.choice(my_headers)})
  if page_html.status_code != 200:
    raise Exception("error")
      #print(page_html.text)

      #解析整个页面获取所有vin对应的urls
      #返回urls
  else:
    print("访问网站成功:",page_html.status_code)

  soup = BeautifulSoup(page_html.content, 'html.parser')
      #print(soup.prettify())  #打印解析后的html

    #下载所有htmls
  url_list = []

  for div in (soup.find("div",  class_ = 'history-box').find_all("div", class_ = 'col-md-3 col-sm-6')):
    url = div.find("a").get("href")
    url_list.append(url)
  print("已获得页面链接")
  #print(url_list)
  return url_list

In [6]:
#解析单页html
#返回
'''
{
  "vincode": "WBAKC810XCDV57019",     #车架号
  "brand": "宝马",                #品牌
  "manufacturer": "宝马(进口)",   #制造厂
  "modelType": "宝马7系",        #车型
  "cc": "4395",                 #发动机排量
  "bodyType": "4门5座三厢车",    #车身型式
  "modelYear": "2011",          #年款
  "engineCode": "N63B44A",      #发动机型号
  "Vehicle_data": [             #对应车款数据
    {"vehicleType": "宝马7系 2011款 750Li xDrive",
    "modelYear": "2011",
    "gearbox": "自动",
    "power": "300",
    "cc": "4395",
    "engineCode": "N63B44A",
    "fuel": "汽油",
    "bodyType": "4门5座三厢车",
    "driveMode": "前置四驱"
    },
    ...,
    {}
    ]
}
'''
def parse_single_html(url):
  html = requests.get(url, headers={'User-Agent':random.choice(my_headers)},)
  if html.status_code != 200:
    print("html download error: ", url)
  else:
    print("download html success: ", url)

  soup2 = BeautifulSoup(html.content, 'html.parser')

  #获取VIN车架号 https://vin17.com/vin/LSVCA6F23P2012225
  vin_data_dict = {}
  arr = url.split("/")
  vincode = arr[-1]

  vin_data_dict['vincode'] = vincode
  #获取WMI/VDS/VIS
  #WMI = soup2.find("span", class_="fir").getText()
  #VDS = soup2.find("span", class_="sec").getText()
  #VIS = soup2.find("span", class_="thi").getText()
  
  #获取汽车详细信息
  vin_data_list = soup2.find("table", class_="table table-bordered table-sm car-table mb-0").find_all("span", attrs={'id':True})
  for vin_data_item in vin_data_list:
    key = vin_data_item.get("id")
    vin_data_dict[key] = vin_data_item.text
  #print(vin_data_dict)
    
  #获取对应车款数据 Vehicle data
  keys_list = ['vehicleType', 'modelYear','gearbox', 'power','cc','engineCode', 'fuel', 'bodyType','driveMode']
  Vehicle_data_list = []
  Vehicle_data_dict = {}
  vehicle_list = soup2.find_all("tr", attrs= {"data-id":True})

  for vehicle_data_tr in vehicle_list:
    data_list = vehicle_data_tr.find_all("td")
    for index, data in enumerate(data_list):
      if index == 9:
        break
      else:
        key = keys_list[index]
        Vehicle_data_dict[key] = data.text.replace('\xa0', ' ')
    #将数据添加到列表中
    Vehicle_data_list.append(Vehicle_data_dict)
  #print(Vehicle_data_list)
  
  vin_data_dict['Vehicle_data'] = Vehicle_data_list

  print("parse success vin: ",vincode)
  return vin_data_dict
  

In [13]:
#测试&存储
def vin_spider_start():
    result_list = []
    data_list = []
    url_list = get_url_list()
    try:
        for url in url_list:
            result_list.append(parse_single_html(url))
            time.sleep(5)

    except Exception as result:
        print(result)
        print("已达到每日限制")
    finally:
        #如何已存在文件
        if os.path.exists('vin_data.json'):
            with open('vin_data.json', 'r', encoding='utf-8') as file:
                str_file = file.read()
                if str_file == 0:
                    print("内容为空")
                else:
                    data_list = json.loads(str_file)
                    #print(data_list)

        data_list.extend(result_list)
        with open('vin_data.json', 'w', encoding='utf-8') as file:
            json.dump(data_list, file, ensure_ascii=False, indent=2)
            print("write data to file success")


In [12]:
vin_spider_start()

访问网站成功: 200
已获得页面链接
download html success:  https://vin17.com/vin/LSVMB6C61MN020908
'NoneType' object has no attribute 'find_all'
已达到每日限制
vindata write to file success


In [14]:
with open('vin_data.json', 'r', encoding='utf-8') as file:
    str_file = file.read()
    if  len(str_file) > 0:
        tmp = json.loads(str_file)
    print(len(tmp))

23
